# Test Regressors, XGBoost, LGB, blending

In [1]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from collections import Counter
import mlflow
import sklearn
from bayes_opt import BayesianOptimization

# local imports
from prepare import *
from evaluate import *

### Set new experiment to keep track in mlflow

In [2]:
mlflow.set_experiment('custom_get_class_pred')
#mlflow.start_run(run_name='run of multiple regressors', nested=True)

### Read in initial datasets if needed

In [3]:
#raw_train, raw_train_labels, raw_test, specs, sample = read_raw_csvs()
#raw_train_labels = pd.read_csv('data/train_labels.csv')

### Load large train/test features from Josh's work

In [4]:
cols_to_drop = ['game_session', 'installation_id', 'accuracy_group']

# Try out balanced classes
reduced_train = pd.read_csv('reduced_train_balanced.csv')
#og_train = pd.read_csv('reduce_train.csv')
#reduced_train = pd.read_csv('reduce_train.csv')
reduced_test = pd.read_csv('reduce_test.csv').drop(cols_to_drop, axis=1)
print(reduced_train.shape, reduced_test.shape)

categoricals = ['session_title']

features = joblib.load('features.pkl')

(35380, 888) (1000, 887)


# Regressors 

In [5]:
from sklearn.ensemble import (
    RandomForestRegressor,
    AdaBoostRegressor,
    GradientBoostingRegressor,
    StackingRegressor,
    VotingRegressor,
    BaggingRegressor,
)

from sklearn.linear_model import (
    HuberRegressor,
    ARDRegression,
    ModifiedHuber,
    PassiveAggressiveRegressor,
    SGDRegressor,
)

from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\imp.py:343: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return _load(spec)


In [6]:
rfr = RandomForestRegressor()
abr = AdaBoostRegressor()
lgbm = LGBMRegressor()
gbr = GradientBoostingRegressor()
hr = HuberRegressor(max_iter=300)
sgd = SGDRegressor()
et = ExtraTreeRegressor()
cbr = CatBoostRegressor(
    loss_function="RMSE",
    task_type="CPU",
    learning_rate=0.05,
    iterations=2000,
    od_type="Iter",
    early_stopping_rounds=500,
    random_seed=42,
)

estimators = [
    #("ABR", abr),
    ("LGBM", lgbm),
    #("GBR", gbr),
    #("HR", hr),
    #("SGD", sgd),  #did poorly with balanced classes
    #("ET", et),    #did poorly with balanced classes
    ("CBR", cbr),
    #("RFR", rfr),
]

In [ ]:
cohen_kappa_score()

In [7]:
for i in estimators:
    print(i[0])
    cv_reg(i[1], reduced_train, reduced_test, n_splits=4)

LGBM
dist={0: 576, 1: 353, 2: 470, 3: 2752}
dist.values=4151
{0: 0.13876174415803422, 1: 0.08503974945796193, 2: 0.11322572874006263, 3: 0.6629727776439412}
acum=0
acum=0.13876174415803422
{0: 1.5062330055279793}
acum=0.13876174415803422
acum=0.22380149361599616
{0: 1.5062330055279793, 1: 1.6929441409456585}
acum=0.22380149361599616
acum=0.3370272223560588
{0: 1.5062330055279793, 1: 1.6929441409456585, 2: 1.8474259802610244}
----------------------------------------
{8: {0: 1.5062330055279793, 1: 1.6929441409456585, 2: 1.8474259802610244}}
dist={0: 421, 1: 459, 2: 630, 3: 2545}
dist.values=4055
{0: 0.10382244143033292, 1: 0.11319358816276202, 2: 0.15536374845869297, 3: 0.6276202219482121}
acum=0
acum=0.10382244143033292
{0: 1.3304913194968266}
acum=0.10382244143033292
acum=0.21701602959309496
{0: 1.3304913194968266, 1: 1.5864719561440745}
acum=0.21701602959309496
acum=0.3723797780517879
{0: 1.3304913194968266, 1: 1.5864719561440745, 2: 1.7900740759787297}
-------------------------------

KeyError: (34.0, 'occurred at index 17698')

In [ ]:
lgbm = quick_eval(reduced_train, lgbm, pc=True)
sub = make_submission(lgbm.predict(reduced_test), og_train)
sub.accuracy_group.value_counts()

In [29]:
cbr = quick_eval(reduced_train, cbr, pc=True)
#sub = make_submission(cbr.predict(reduced_test), reduced_train)
#sub.accuracy_group.value_counts()

0:	learn: 1.1040526	total: 69.5ms	remaining: 2m 18s
1:	learn: 1.0921305	total: 128ms	remaining: 2m 7s
2:	learn: 1.0810385	total: 199ms	remaining: 2m 12s
3:	learn: 1.0695510	total: 266ms	remaining: 2m 12s
4:	learn: 1.0597420	total: 342ms	remaining: 2m 16s
5:	learn: 1.0508257	total: 396ms	remaining: 2m 11s
6:	learn: 1.0428121	total: 453ms	remaining: 2m 9s
7:	learn: 1.0357688	total: 518ms	remaining: 2m 8s
8:	learn: 1.0285120	total: 574ms	remaining: 2m 6s
9:	learn: 1.0221902	total: 628ms	remaining: 2m 4s
10:	learn: 1.0156879	total: 688ms	remaining: 2m 4s
11:	learn: 1.0101707	total: 747ms	remaining: 2m 3s
12:	learn: 1.0041668	total: 807ms	remaining: 2m 3s
13:	learn: 0.9987999	total: 864ms	remaining: 2m 2s
14:	learn: 0.9945186	total: 920ms	remaining: 2m 1s
15:	learn: 0.9898010	total: 977ms	remaining: 2m 1s
16:	learn: 0.9862048	total: 1.03s	remaining: 2m
17:	learn: 0.9824887	total: 1.08s	remaining: 1m 59s
18:	learn: 0.9790405	total: 1.14s	remaining: 1m 59s
19:	learn: 0.9758592	total: 1.2s	rem

161:	learn: 0.8695938	total: 9.71s	remaining: 1m 50s
162:	learn: 0.8693314	total: 9.77s	remaining: 1m 50s
163:	learn: 0.8691410	total: 9.82s	remaining: 1m 49s
164:	learn: 0.8688837	total: 9.88s	remaining: 1m 49s
165:	learn: 0.8686436	total: 9.94s	remaining: 1m 49s
166:	learn: 0.8681475	total: 10s	remaining: 1m 49s
167:	learn: 0.8680541	total: 10.1s	remaining: 1m 49s
168:	learn: 0.8678449	total: 10.1s	remaining: 1m 49s
169:	learn: 0.8674703	total: 10.2s	remaining: 1m 49s
170:	learn: 0.8671707	total: 10.2s	remaining: 1m 49s
171:	learn: 0.8670113	total: 10.3s	remaining: 1m 49s
172:	learn: 0.8666369	total: 10.4s	remaining: 1m 49s
173:	learn: 0.8663888	total: 10.4s	remaining: 1m 49s
174:	learn: 0.8659980	total: 10.5s	remaining: 1m 49s
175:	learn: 0.8658504	total: 10.5s	remaining: 1m 49s
176:	learn: 0.8656842	total: 10.6s	remaining: 1m 48s
177:	learn: 0.8655441	total: 10.6s	remaining: 1m 48s
178:	learn: 0.8650653	total: 10.7s	remaining: 1m 48s
179:	learn: 0.8649216	total: 10.7s	remaining: 1m

319:	learn: 0.8262669	total: 18.6s	remaining: 1m 37s
320:	learn: 0.8260950	total: 18.7s	remaining: 1m 37s
321:	learn: 0.8258297	total: 18.7s	remaining: 1m 37s
322:	learn: 0.8255852	total: 18.8s	remaining: 1m 37s
323:	learn: 0.8253737	total: 18.8s	remaining: 1m 37s
324:	learn: 0.8251007	total: 18.9s	remaining: 1m 37s
325:	learn: 0.8248306	total: 18.9s	remaining: 1m 37s
326:	learn: 0.8246421	total: 19s	remaining: 1m 37s
327:	learn: 0.8244231	total: 19.1s	remaining: 1m 37s
328:	learn: 0.8241068	total: 19.1s	remaining: 1m 37s
329:	learn: 0.8238573	total: 19.2s	remaining: 1m 36s
330:	learn: 0.8237345	total: 19.2s	remaining: 1m 36s
331:	learn: 0.8235496	total: 19.3s	remaining: 1m 36s
332:	learn: 0.8232294	total: 19.3s	remaining: 1m 36s
333:	learn: 0.8229510	total: 19.4s	remaining: 1m 36s
334:	learn: 0.8228018	total: 19.4s	remaining: 1m 36s
335:	learn: 0.8225805	total: 19.5s	remaining: 1m 36s
336:	learn: 0.8225553	total: 19.5s	remaining: 1m 36s
337:	learn: 0.8223741	total: 19.6s	remaining: 1m

475:	learn: 0.7959067	total: 28.5s	remaining: 1m 31s
476:	learn: 0.7958942	total: 28.5s	remaining: 1m 31s
477:	learn: 0.7956640	total: 28.6s	remaining: 1m 31s
478:	learn: 0.7954884	total: 28.7s	remaining: 1m 31s
479:	learn: 0.7953010	total: 28.7s	remaining: 1m 30s
480:	learn: 0.7951051	total: 28.8s	remaining: 1m 30s
481:	learn: 0.7948708	total: 28.8s	remaining: 1m 30s
482:	learn: 0.7946431	total: 28.9s	remaining: 1m 30s
483:	learn: 0.7944692	total: 28.9s	remaining: 1m 30s
484:	learn: 0.7942321	total: 29s	remaining: 1m 30s
485:	learn: 0.7940926	total: 29.1s	remaining: 1m 30s
486:	learn: 0.7940678	total: 29.1s	remaining: 1m 30s
487:	learn: 0.7939190	total: 29.2s	remaining: 1m 30s
488:	learn: 0.7937115	total: 29.2s	remaining: 1m 30s
489:	learn: 0.7936985	total: 29.3s	remaining: 1m 30s
490:	learn: 0.7934896	total: 29.3s	remaining: 1m 30s
491:	learn: 0.7933405	total: 29.4s	remaining: 1m 30s
492:	learn: 0.7932183	total: 29.4s	remaining: 1m 30s
493:	learn: 0.7930329	total: 29.5s	remaining: 1m

631:	learn: 0.7717417	total: 37.7s	remaining: 1m 21s
632:	learn: 0.7715238	total: 37.7s	remaining: 1m 21s
633:	learn: 0.7713911	total: 37.8s	remaining: 1m 21s
634:	learn: 0.7711414	total: 37.8s	remaining: 1m 21s
635:	learn: 0.7710410	total: 37.9s	remaining: 1m 21s
636:	learn: 0.7709589	total: 38s	remaining: 1m 21s
637:	learn: 0.7707744	total: 38s	remaining: 1m 21s
638:	learn: 0.7706330	total: 38.1s	remaining: 1m 21s
639:	learn: 0.7705512	total: 38.1s	remaining: 1m 21s
640:	learn: 0.7703850	total: 38.2s	remaining: 1m 21s
641:	learn: 0.7702382	total: 38.3s	remaining: 1m 20s
642:	learn: 0.7700798	total: 38.3s	remaining: 1m 20s
643:	learn: 0.7700333	total: 38.4s	remaining: 1m 20s
644:	learn: 0.7699148	total: 38.4s	remaining: 1m 20s
645:	learn: 0.7697262	total: 38.5s	remaining: 1m 20s
646:	learn: 0.7696188	total: 38.5s	remaining: 1m 20s
647:	learn: 0.7694432	total: 38.6s	remaining: 1m 20s
648:	learn: 0.7693124	total: 38.6s	remaining: 1m 20s
649:	learn: 0.7690989	total: 38.7s	remaining: 1m 2

...

1265:	learn: 0.6973761	total: 1m 13s	remaining: 42.4s
1266:	learn: 0.6972305	total: 1m 13s	remaining: 42.4s
1267:	learn: 0.6971177	total: 1m 13s	remaining: 42.3s
1268:	learn: 0.6970270	total: 1m 13s	remaining: 42.2s
1269:	learn: 0.6969144	total: 1m 13s	remaining: 42.2s
1270:	learn: 0.6967449	total: 1m 13s	remaining: 42.1s
1271:	learn: 0.6966354	total: 1m 13s	remaining: 42.1s
1272:	learn: 0.6965881	total: 1m 13s	remaining: 42s
1273:	learn: 0.6964970	total: 1m 13s	remaining: 42s
1274:	learn: 0.6963726	total: 1m 13s	remaining: 41.9s
1275:	learn: 0.6962808	total: 1m 13s	remaining: 41.8s
1276:	learn: 0.6961218	total: 1m 13s	remaining: 41.8s
1277:	learn: 0.6959946	total: 1m 13s	remaining: 41.7s
1278:	learn: 0.6958829	total: 1m 13s	remaining: 41.7s
1279:	learn: 0.6958645	total: 1m 13s	remaining: 41.6s
1280:	learn: 0.6957250	total: 1m 14s	remaining: 41.5s
1281:	learn: 0.6956417	total: 1m 14s	remaining: 41.5s
1282:	learn: 0.6955776	total: 1m 14s	remaining: 41.4s
1283:	learn: 0.6954682	total: 1m

1419:	learn: 0.6832092	total: 1m 22s	remaining: 33.6s
1420:	learn: 0.6831373	total: 1m 22s	remaining: 33.6s
1421:	learn: 0.6830817	total: 1m 22s	remaining: 33.5s
1422:	learn: 0.6829610	total: 1m 22s	remaining: 33.5s
1423:	learn: 0.6828465	total: 1m 22s	remaining: 33.4s
1424:	learn: 0.6827526	total: 1m 22s	remaining: 33.3s
1425:	learn: 0.6826162	total: 1m 22s	remaining: 33.3s
1426:	learn: 0.6825593	total: 1m 22s	remaining: 33.2s
1427:	learn: 0.6824667	total: 1m 22s	remaining: 33.2s
1428:	learn: 0.6824504	total: 1m 22s	remaining: 33.1s
1429:	learn: 0.6823168	total: 1m 22s	remaining: 33.1s
1430:	learn: 0.6822200	total: 1m 22s	remaining: 33s
1431:	learn: 0.6821150	total: 1m 23s	remaining: 32.9s
1432:	learn: 0.6819359	total: 1m 23s	remaining: 32.9s
1433:	learn: 0.6818367	total: 1m 23s	remaining: 32.8s
1434:	learn: 0.6817301	total: 1m 23s	remaining: 32.8s
1435:	learn: 0.6816087	total: 1m 23s	remaining: 32.7s
1436:	learn: 0.6815165	total: 1m 23s	remaining: 32.7s
1437:	learn: 0.6815118	total: 

1573:	learn: 0.6688979	total: 1m 34s	remaining: 25.6s
1574:	learn: 0.6687911	total: 1m 34s	remaining: 25.5s
1575:	learn: 0.6686829	total: 1m 34s	remaining: 25.5s
1576:	learn: 0.6685772	total: 1m 34s	remaining: 25.4s
1577:	learn: 0.6684624	total: 1m 34s	remaining: 25.4s
1578:	learn: 0.6683617	total: 1m 34s	remaining: 25.3s
1579:	learn: 0.6682371	total: 1m 34s	remaining: 25.2s
1580:	learn: 0.6681847	total: 1m 35s	remaining: 25.2s
1581:	learn: 0.6680949	total: 1m 35s	remaining: 25.1s
1582:	learn: 0.6680115	total: 1m 35s	remaining: 25.1s
1583:	learn: 0.6678846	total: 1m 35s	remaining: 25s
1584:	learn: 0.6677946	total: 1m 35s	remaining: 25s
1585:	learn: 0.6677229	total: 1m 35s	remaining: 24.9s
1586:	learn: 0.6676074	total: 1m 35s	remaining: 24.9s
1587:	learn: 0.6674878	total: 1m 35s	remaining: 24.8s
1588:	learn: 0.6674335	total: 1m 35s	remaining: 24.7s
1589:	learn: 0.6673407	total: 1m 35s	remaining: 24.7s
1590:	learn: 0.6672508	total: 1m 35s	remaining: 24.6s
1591:	learn: 0.6671948	total: 1m

1726:	learn: 0.6555679	total: 1m 46s	remaining: 16.9s
1727:	learn: 0.6554647	total: 1m 46s	remaining: 16.8s
1728:	learn: 0.6554146	total: 1m 47s	remaining: 16.8s
1729:	learn: 0.6553706	total: 1m 47s	remaining: 16.7s
1730:	learn: 0.6552611	total: 1m 47s	remaining: 16.7s
1731:	learn: 0.6551825	total: 1m 47s	remaining: 16.6s
1732:	learn: 0.6550965	total: 1m 47s	remaining: 16.5s
1733:	learn: 0.6549797	total: 1m 47s	remaining: 16.5s
1734:	learn: 0.6549113	total: 1m 47s	remaining: 16.4s
1735:	learn: 0.6548554	total: 1m 47s	remaining: 16.4s
1736:	learn: 0.6548017	total: 1m 47s	remaining: 16.3s
1737:	learn: 0.6547444	total: 1m 47s	remaining: 16.2s
1738:	learn: 0.6546495	total: 1m 47s	remaining: 16.2s
1739:	learn: 0.6546073	total: 1m 47s	remaining: 16.1s
1740:	learn: 0.6545515	total: 1m 48s	remaining: 16.1s
1741:	learn: 0.6544516	total: 1m 48s	remaining: 16s
1742:	learn: 0.6542843	total: 1m 48s	remaining: 15.9s
1743:	learn: 0.6542719	total: 1m 48s	remaining: 15.9s
1744:	learn: 0.6541692	total: 

1881:	learn: 0.6428423	total: 1m 58s	remaining: 7.45s
1882:	learn: 0.6427751	total: 1m 58s	remaining: 7.39s
1883:	learn: 0.6427069	total: 1m 58s	remaining: 7.33s
1884:	learn: 0.6426637	total: 1m 59s	remaining: 7.26s
1885:	learn: 0.6425727	total: 1m 59s	remaining: 7.2s
1886:	learn: 0.6424824	total: 1m 59s	remaining: 7.13s
1887:	learn: 0.6423749	total: 1m 59s	remaining: 7.07s
1888:	learn: 0.6422650	total: 1m 59s	remaining: 7.01s
1889:	learn: 0.6421910	total: 1m 59s	remaining: 6.95s
1890:	learn: 0.6421627	total: 1m 59s	remaining: 6.88s
1891:	learn: 0.6421293	total: 1m 59s	remaining: 6.82s
1892:	learn: 0.6420316	total: 1m 59s	remaining: 6.75s
1893:	learn: 0.6419693	total: 1m 59s	remaining: 6.69s
1894:	learn: 0.6419651	total: 1m 59s	remaining: 6.63s
1895:	learn: 0.6418889	total: 1m 59s	remaining: 6.57s
1896:	learn: 0.6418194	total: 1m 59s	remaining: 6.5s
1897:	learn: 0.6417830	total: 1m 59s	remaining: 6.44s
1898:	learn: 0.6416539	total: 1m 59s	remaining: 6.38s
1899:	learn: 0.6415427	total: 

In [30]:
cv_reg(cbr, reduced_train, n_splits=8)

0:	learn: 1.1047878	total: 85.8ms	remaining: 2m 51s
1:	learn: 1.0923338	total: 168ms	remaining: 2m 47s
2:	learn: 1.0808992	total: 246ms	remaining: 2m 43s
3:	learn: 1.0694924	total: 336ms	remaining: 2m 47s
4:	learn: 1.0591137	total: 414ms	remaining: 2m 45s
5:	learn: 1.0498043	total: 496ms	remaining: 2m 44s
6:	learn: 1.0409630	total: 593ms	remaining: 2m 48s
7:	learn: 1.0333647	total: 670ms	remaining: 2m 46s
8:	learn: 1.0260203	total: 741ms	remaining: 2m 43s
9:	learn: 1.0192523	total: 821ms	remaining: 2m 43s
10:	learn: 1.0132152	total: 958ms	remaining: 2m 53s
11:	learn: 1.0071759	total: 1.25s	remaining: 3m 26s
12:	learn: 1.0012415	total: 1.43s	remaining: 3m 39s
13:	learn: 0.9959447	total: 1.52s	remaining: 3m 35s
14:	learn: 0.9917570	total: 1.6s	remaining: 3m 32s
15:	learn: 0.9869320	total: 1.7s	remaining: 3m 31s
16:	learn: 0.9832515	total: 1.79s	remaining: 3m 28s
17:	learn: 0.9789708	total: 1.87s	remaining: 3m 26s
18:	learn: 0.9751120	total: 1.96s	remaining: 3m 24s
19:	learn: 0.9714166	to

159:	learn: 0.8688116	total: 14.4s	remaining: 2m 46s
160:	learn: 0.8686029	total: 14.5s	remaining: 2m 45s
161:	learn: 0.8684840	total: 14.6s	remaining: 2m 45s
162:	learn: 0.8681773	total: 14.7s	remaining: 2m 45s
163:	learn: 0.8678777	total: 14.7s	remaining: 2m 45s
164:	learn: 0.8677195	total: 14.8s	remaining: 2m 44s
165:	learn: 0.8673529	total: 14.9s	remaining: 2m 44s
166:	learn: 0.8671147	total: 15s	remaining: 2m 44s
167:	learn: 0.8668976	total: 15s	remaining: 2m 43s
168:	learn: 0.8665679	total: 15.1s	remaining: 2m 43s
169:	learn: 0.8663260	total: 15.2s	remaining: 2m 43s
170:	learn: 0.8659257	total: 15.3s	remaining: 2m 43s
171:	learn: 0.8658311	total: 15.3s	remaining: 2m 42s
172:	learn: 0.8656845	total: 15.4s	remaining: 2m 42s
173:	learn: 0.8655477	total: 15.5s	remaining: 2m 42s
174:	learn: 0.8652601	total: 15.5s	remaining: 2m 41s
175:	learn: 0.8649880	total: 15.6s	remaining: 2m 41s
176:	learn: 0.8647189	total: 15.7s	remaining: 2m 41s
177:	learn: 0.8643692	total: 15.8s	remaining: 2m 4

315:	learn: 0.8290535	total: 25.8s	remaining: 2m 17s
316:	learn: 0.8288231	total: 25.9s	remaining: 2m 17s
317:	learn: 0.8285620	total: 26s	remaining: 2m 17s
318:	learn: 0.8283391	total: 26s	remaining: 2m 17s
319:	learn: 0.8281170	total: 26.1s	remaining: 2m 17s
320:	learn: 0.8279428	total: 26.2s	remaining: 2m 16s
321:	learn: 0.8277367	total: 26.2s	remaining: 2m 16s
322:	learn: 0.8275738	total: 26.3s	remaining: 2m 16s
323:	learn: 0.8272741	total: 26.4s	remaining: 2m 16s
324:	learn: 0.8270684	total: 26.5s	remaining: 2m 16s
325:	learn: 0.8268722	total: 26.5s	remaining: 2m 16s
326:	learn: 0.8268110	total: 26.6s	remaining: 2m 16s
327:	learn: 0.8265252	total: 26.7s	remaining: 2m 15s
328:	learn: 0.8263568	total: 26.7s	remaining: 2m 15s
329:	learn: 0.8260770	total: 26.8s	remaining: 2m 15s
330:	learn: 0.8258696	total: 26.9s	remaining: 2m 15s
331:	learn: 0.8257040	total: 26.9s	remaining: 2m 15s
332:	learn: 0.8254973	total: 27s	remaining: 2m 15s
333:	learn: 0.8251528	total: 27.1s	remaining: 2m 15s

475:	learn: 0.7997735	total: 37.3s	remaining: 1m 59s
476:	learn: 0.7995802	total: 37.4s	remaining: 1m 59s
477:	learn: 0.7994042	total: 37.5s	remaining: 1m 59s
478:	learn: 0.7991525	total: 37.5s	remaining: 1m 59s
479:	learn: 0.7990044	total: 37.6s	remaining: 1m 59s
480:	learn: 0.7989958	total: 37.6s	remaining: 1m 58s
481:	learn: 0.7987720	total: 37.7s	remaining: 1m 58s
482:	learn: 0.7985684	total: 37.8s	remaining: 1m 58s
483:	learn: 0.7983707	total: 37.8s	remaining: 1m 58s
484:	learn: 0.7981478	total: 37.9s	remaining: 1m 58s
485:	learn: 0.7980595	total: 38s	remaining: 1m 58s
486:	learn: 0.7978919	total: 38s	remaining: 1m 58s
487:	learn: 0.7977372	total: 38.1s	remaining: 1m 57s
488:	learn: 0.7975290	total: 38.1s	remaining: 1m 57s
489:	learn: 0.7973588	total: 38.2s	remaining: 1m 57s
490:	learn: 0.7972114	total: 38.3s	remaining: 1m 57s
491:	learn: 0.7970249	total: 38.3s	remaining: 1m 57s
492:	learn: 0.7968078	total: 38.4s	remaining: 1m 57s
493:	learn: 0.7966288	total: 38.5s	remaining: 1m 5

633:	learn: 0.7751754	total: 48.6s	remaining: 1m 44s
634:	learn: 0.7749716	total: 48.7s	remaining: 1m 44s
635:	learn: 0.7748082	total: 48.8s	remaining: 1m 44s
636:	learn: 0.7746930	total: 48.8s	remaining: 1m 44s
637:	learn: 0.7745059	total: 48.9s	remaining: 1m 44s
638:	learn: 0.7743619	total: 49s	remaining: 1m 44s
639:	learn: 0.7741653	total: 49.1s	remaining: 1m 44s
640:	learn: 0.7739879	total: 49.2s	remaining: 1m 44s
641:	learn: 0.7738693	total: 49.2s	remaining: 1m 44s
642:	learn: 0.7737211	total: 49.4s	remaining: 1m 44s
643:	learn: 0.7735348	total: 49.7s	remaining: 1m 44s
644:	learn: 0.7733915	total: 49.8s	remaining: 1m 44s
645:	learn: 0.7732346	total: 49.9s	remaining: 1m 44s
646:	learn: 0.7730419	total: 50s	remaining: 1m 44s
647:	learn: 0.7728463	total: 50.1s	remaining: 1m 44s
648:	learn: 0.7726970	total: 50.2s	remaining: 1m 44s
649:	learn: 0.7724449	total: 50.2s	remaining: 1m 44s
650:	learn: 0.7722803	total: 50.3s	remaining: 1m 44s
651:	learn: 0.7721893	total: 50.4s	remaining: 1m 4

...

1338:	learn: 0.6940957	total: 1m 10s	remaining: 34.8s
1339:	learn: 0.6940077	total: 1m 10s	remaining: 34.7s
1340:	learn: 0.6939100	total: 1m 10s	remaining: 34.7s
1341:	learn: 0.6937775	total: 1m 10s	remaining: 34.6s
1342:	learn: 0.6936962	total: 1m 10s	remaining: 34.6s
1343:	learn: 0.6936184	total: 1m 10s	remaining: 34.5s
1344:	learn: 0.6935512	total: 1m 10s	remaining: 34.5s
1345:	learn: 0.6934174	total: 1m 10s	remaining: 34.4s
1346:	learn: 0.6933727	total: 1m 10s	remaining: 34.4s
1347:	learn: 0.6933574	total: 1m 10s	remaining: 34.3s
1348:	learn: 0.6933547	total: 1m 10s	remaining: 34.2s
1349:	learn: 0.6932478	total: 1m 11s	remaining: 34.2s
1350:	learn: 0.6931345	total: 1m 11s	remaining: 34.1s
1351:	learn: 0.6930149	total: 1m 11s	remaining: 34.1s
1352:	learn: 0.6929533	total: 1m 11s	remaining: 34s
1353:	learn: 0.6928390	total: 1m 11s	remaining: 34s
1354:	learn: 0.6927256	total: 1m 11s	remaining: 33.9s
1355:	learn: 0.6926538	total: 1m 11s	remaining: 33.9s
1356:	learn: 0.6925818	total: 1m

1491:	learn: 0.6803298	total: 1m 18s	remaining: 26.7s
1492:	learn: 0.6801958	total: 1m 18s	remaining: 26.7s
1493:	learn: 0.6800990	total: 1m 18s	remaining: 26.6s
1494:	learn: 0.6800158	total: 1m 18s	remaining: 26.6s
1495:	learn: 0.6799491	total: 1m 18s	remaining: 26.5s
1496:	learn: 0.6798356	total: 1m 18s	remaining: 26.5s
1497:	learn: 0.6797425	total: 1m 18s	remaining: 26.4s
1498:	learn: 0.6796358	total: 1m 18s	remaining: 26.4s
1499:	learn: 0.6795077	total: 1m 18s	remaining: 26.3s
1500:	learn: 0.6793889	total: 1m 19s	remaining: 26.3s
1501:	learn: 0.6792723	total: 1m 19s	remaining: 26.2s
1502:	learn: 0.6791959	total: 1m 19s	remaining: 26.2s
1503:	learn: 0.6791169	total: 1m 19s	remaining: 26.1s
1504:	learn: 0.6790409	total: 1m 19s	remaining: 26.1s
1505:	learn: 0.6789880	total: 1m 19s	remaining: 26s
1506:	learn: 0.6788768	total: 1m 19s	remaining: 26s
1507:	learn: 0.6788489	total: 1m 19s	remaining: 25.9s
1508:	learn: 0.6787593	total: 1m 19s	remaining: 25.9s
1509:	learn: 0.6787461	total: 1m

1644:	learn: 0.6671337	total: 1m 26s	remaining: 18.7s
1645:	learn: 0.6670618	total: 1m 26s	remaining: 18.7s
1646:	learn: 0.6669999	total: 1m 26s	remaining: 18.6s
1647:	learn: 0.6668980	total: 1m 26s	remaining: 18.6s
1648:	learn: 0.6667740	total: 1m 26s	remaining: 18.5s
1649:	learn: 0.6666825	total: 1m 26s	remaining: 18.4s
1650:	learn: 0.6665607	total: 1m 27s	remaining: 18.4s
1651:	learn: 0.6665033	total: 1m 27s	remaining: 18.3s
1652:	learn: 0.6663820	total: 1m 27s	remaining: 18.3s
1653:	learn: 0.6663249	total: 1m 27s	remaining: 18.2s
1654:	learn: 0.6661975	total: 1m 27s	remaining: 18.2s
1655:	learn: 0.6660962	total: 1m 27s	remaining: 18.1s
1656:	learn: 0.6660628	total: 1m 27s	remaining: 18.1s
1657:	learn: 0.6659284	total: 1m 27s	remaining: 18s
1658:	learn: 0.6657448	total: 1m 27s	remaining: 18s
1659:	learn: 0.6656667	total: 1m 27s	remaining: 17.9s
1660:	learn: 0.6655857	total: 1m 27s	remaining: 17.9s
1661:	learn: 0.6655296	total: 1m 27s	remaining: 17.8s
1662:	learn: 0.6654135	total: 1m

1800:	learn: 0.6539250	total: 1m 34s	remaining: 10.5s
1801:	learn: 0.6538239	total: 1m 34s	remaining: 10.4s
1802:	learn: 0.6537736	total: 1m 34s	remaining: 10.3s
1803:	learn: 0.6536686	total: 1m 34s	remaining: 10.3s
1804:	learn: 0.6535451	total: 1m 34s	remaining: 10.2s
1805:	learn: 0.6534663	total: 1m 34s	remaining: 10.2s
1806:	learn: 0.6533784	total: 1m 34s	remaining: 10.1s
1807:	learn: 0.6533418	total: 1m 34s	remaining: 10.1s
1808:	learn: 0.6532517	total: 1m 34s	remaining: 10s
1809:	learn: 0.6531719	total: 1m 35s	remaining: 9.98s
1810:	learn: 0.6531046	total: 1m 35s	remaining: 9.92s
1811:	learn: 0.6530547	total: 1m 35s	remaining: 9.87s
1812:	learn: 0.6529881	total: 1m 35s	remaining: 9.82s
1813:	learn: 0.6529564	total: 1m 35s	remaining: 9.76s
1814:	learn: 0.6528761	total: 1m 35s	remaining: 9.71s
1815:	learn: 0.6527776	total: 1m 35s	remaining: 9.66s
1816:	learn: 0.6526697	total: 1m 35s	remaining: 9.61s
1817:	learn: 0.6525731	total: 1m 35s	remaining: 9.55s
1818:	learn: 0.6524861	total: 

1952:	learn: 0.6418596	total: 1m 42s	remaining: 2.46s
1953:	learn: 0.6418024	total: 1m 42s	remaining: 2.41s
1954:	learn: 0.6417698	total: 1m 42s	remaining: 2.36s
1955:	learn: 0.6417000	total: 1m 42s	remaining: 2.31s
1956:	learn: 0.6416286	total: 1m 42s	remaining: 2.25s
1957:	learn: 0.6415266	total: 1m 42s	remaining: 2.2s
1958:	learn: 0.6414243	total: 1m 42s	remaining: 2.15s
1959:	learn: 0.6413669	total: 1m 42s	remaining: 2.1s
1960:	learn: 0.6413251	total: 1m 42s	remaining: 2.04s
1961:	learn: 0.6412233	total: 1m 42s	remaining: 1.99s
1962:	learn: 0.6411522	total: 1m 42s	remaining: 1.94s
1963:	learn: 0.6410839	total: 1m 42s	remaining: 1.89s
1964:	learn: 0.6409961	total: 1m 43s	remaining: 1.83s
1965:	learn: 0.6409747	total: 1m 43s	remaining: 1.78s
1966:	learn: 0.6409018	total: 1m 43s	remaining: 1.73s
1967:	learn: 0.6408170	total: 1m 43s	remaining: 1.68s
1968:	learn: 0.6407588	total: 1m 43s	remaining: 1.63s
1969:	learn: 0.6406647	total: 1m 43s	remaining: 1.57s
1970:	learn: 0.6406624	total: 

In [16]:
bag_reg = BaggingRegressor(base_estimator=lgbm,
                      n_estimators=10, random_state=42)
bag_reg = quick_eval(reduced_train, bag_reg, pc=True)
cv_reg(bag_reg, reduced_train, n_splits=5)

The accuracy of BaggingRegressor is 0.539711701526286
The QWK of BaggingRegressor is 0.6490823105506136
1/5 Fold start| QWK=0.6464777602167533
2/5 Fold start| QWK=0.63639959317437
3/5 Fold start| QWK=0.6470655141037307
4/5 Fold start| QWK=0.6293416572077185
5/5 Fold start| QWK=0.6529273216689098


MlflowException: Param value 'LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estim' had length 452, which exceeded length limit of 250

## Tune LGBM

## Vote, stacked, bagged regressors

In [40]:
vote_regressor = VotingRegressor(estimators=estimators,
                                n_jobs=-1)
vote_regressor = quick_eval(reduced_train, vote_regressor, pc=True)

The accuracy of VotingRegressor is 0.5712266817410967
The QWK of VotingRegressor is 0.5915220967303223


In [53]:
stacked_regressor = StackingRegressor(estimators=estimators,
                                n_jobs=-1)
stacked_regressor = quick_eval(reduced_train, stacked_regressor, pc=True)

The accuracy of StackingRegressor is 0.5672696438665913
The QWK of StackingRegressor is 0.5979707068817519


In [55]:
bagged_regressor = BaggingRegressor(base_estimator=lgbm,
                               n_jobs=-1,
                                   warm_start=True)
quick_eval(reduced_train, bagged_regressor, pc=True)

The accuracy of BaggingRegressor is 0.5746184284906727
The QWK of BaggingRegressor is 0.5958131257022264


BaggingRegressor(base_estimator=LGBMRegressor(boosting_type='gbdt',
                                              class_weight=None,
                                              colsample_bytree=1.0,
                                              importance_type='split',
                                              learning_rate=0.1, max_depth=-1,
                                              min_child_samples=20,
                                              min_child_weight=0.001,
                                              min_split_gain=0.0,
                                              n_estimators=100, n_jobs=-1,
                                              num_leaves=31, objective=None,
                                              random_state=None, reg_alpha=0.0,
                                              reg_lambda=0.0, silent=True,
                                              subsample=1.0,
                                              subsample_for_bin=200000,
     

##### Grid Search/Random Search

In [39]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold

In [69]:
#grid of parameters
gridParams = {
    'learning_rate': [0.05],
    'num_leaves': [31,90,200],
    'boosting_type' : ['gbdt','dart','rf'],
    'objective' : ['regression'],
    'max_depth' : [5,6,7,8],
    'random_state' : [42], 
    'colsample_bytree' : [0.3,0.5,0.7],
    'subsample' : [0.3,0.5,0.7],
    'min_split_gain' : [0.01],
    'min_data_in_leaf':[10],
    'metric':['rmse']
    }
#modelling
lgbm = LGBMRegressor()
grid = RandomizedSearchCV(lgbm,gridParams,cv=10,n_jobs = -1,n_iter=10)
X = reduced_train.drop(cols_to_drop, axis=1)[features]
y = reduced_train.accuracy_group

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
        
grid.fit(X_train,y_train)

grid.best_params_
lgbm = grid.best_estimator_.fit(X_train, y_train)
y_pred = get_class_pred(
    grid.predict(X_test),
    reduced_train)
accuracy = accuracy_score(y_test, y_pred)
qwk = cohen_kappa_score(y_test, y_pred, weights="quadratic")
mlflow.log_param("features_shape", X.shape)
mlflow.log_param("estimator", 'xgb')
mlflow.log_metric("Accuracy", accuracy)
mlflow.log_metric("QWK", qwk)
print(accuracy, qwk)

0.5712266817410967 0.5988696990398876


## Manual crossval

In [47]:
kf = KFold()

C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


## XGBoost

In [63]:
mlflow.start_run(run_name='xgb-tuned-params', nested=True)


import xgboost as xgb
from sklearn import datasets
from sklearn.model_selection import train_test_split


X = reduced_train.drop(cols_to_drop, axis=1)[features]
y = reduced_train.accuracy_group

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

dtrain = xgb.DMatrix(X_train, label=y_train)
params = {
            'colsample_bytree': 0.2,                 
            'learning_rate': 0.01,
            'objective':'reg:squarederror',
            'max_depth': 6,
            'subsample': 1,
            'min_child_weight': 3,
            'gamma': 0.25,
            'n_estimators': 1400
         }
xgb_model = xgb.train(params=params, dtrain=dtrain, num_boost_round=10)

xgb_model_path = "xgb_model.pth"
xgb_model.save_model(xgb_model_path)

dtest = xgb.DMatrix(X_test, label=y_test)

print('through the test')
y_pred = xgb_model.predict(dtest)

y_pred = get_class_pred(y_pred,reduced_train)

print('through the test')

accuracy = accuracy_score(y_test, y_pred)
qwk = cohen_kappa_score(y_test, y_pred, weights="quadratic")
mlflow.log_param("features_shape", X.shape)
mlflow.log_param("estimator", 'xgb')
mlflow.log_metric("Accuracy", accuracy)
mlflow.log_metric("QWK", qwk)
mlflow.sklearn.log_model(xgb_model, "model")
mlflow.end_run()
print('complete')

through the test
through the test
complete
